AUTHOR : TITUS OLANG'

INTRO   : THIS IS A PREDICT BASED ON SOUTH AFRICAN LANGUAGES. SOUTH AFRICA IS KNOWN TO HAVE 11 DISTINCT LANGUAGES.


PROBLEM STATEMENT : THE PROJECT LOOKS TO USE NATURAL LANGUAGE PROCESSING TO HELP DETECT WHAT SOUTH AFRICAN LANGUAGE WAS IN THE TEXT


In [1]:
##IMPORT LIBRARIES

import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")
# data preprocessing

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

# classification model
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.naive_bayes import MultinomialNB


# Performance Evaluation

from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score


In [2]:
##DATA LOADING
train_df = pd.read_csv("train_set.csv")
test_df=pd.read_csv('test_set.csv')

## Data preprocessing

In [3]:
#Check for a subset of the data
train_df.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [4]:
train_df['lang_id'].unique() #checking the number of language the data has

array(['xho', 'eng', 'nso', 'ven', 'tsn', 'nbl', 'zul', 'ssw', 'tso',
       'sot', 'afr'], dtype=object)

In [5]:
#Check for the shape
train_df.shape

(33000, 2)

In [6]:
#Check for the datatype 
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33000 entries, 0 to 32999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   lang_id  33000 non-null  object
 1   text     33000 non-null  object
dtypes: object(2)
memory usage: 515.8+ KB


In [7]:
train_df["lang_id"].value_counts() #count the number of centans per language

xho    3000
eng    3000
nso    3000
ven    3000
tsn    3000
nbl    3000
zul    3000
ssw    3000
tso    3000
sot    3000
afr    3000
Name: lang_id, dtype: int64

In [8]:
#Separating Independent and Dependent variables

X = train_df["text"]
y = train_df["lang_id"]

In [9]:
# creating list for appending the preprocessed text
data_list = []
for text in X:
        text = re.sub(r'[!@#$(),n"%^*?:;~`0-9]', ' ', text)
        text = re.sub(r'[[]]', ' ', text)
        text = text.lower()
        data_list.append(text)

In [10]:
# Text preprocessing for test data
data_list = []
for text in test_df['text']:
        text = re.sub(r'[!@#$(),n"%^*?:;~`0-9]', ' ', text)
        text = re.sub(r'[[]]', ' ', text)
        text = text.lower()
        data_list.append(text)

## MODELING

In [11]:
# Splitting the data into the training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [12]:
#Label encoding
vect = TfidfVectorizer()
vect.fit_transform(X_train)

<26400x125177 sparse matrix of type '<class 'numpy.float64'>'
	with 744909 stored elements in Compressed Sparse Row format>

### Trying different models

In [13]:
#Model Training and Prediction
svc = LinearSVC()
svc.fit(vect.transform(X_train), y_train)
y_pred = svc.predict(vect.transform(X_test))

In [14]:
#Instatiating the Tfidf vectorizer
tfidf_vect = TfidfVectorizer(ngram_range=(1, 2),min_df = 2, max_features = 5000)

In [15]:
#Transforming to numeric
X_train = tfidf_vect.fit_transform(X_train).toarray() 
X_test = tfidf_vect.fit_transform(X_test).toarray() 

In [16]:
lreg = LogisticRegression()
lreg.fit(X_train, y_train)

LogisticRegression()

In [17]:
# LogisticRegression
y_pred_test = lreg.predict(X_test)
y_pred_train = lreg.predict(X_train)
f1score_train = f1_score(y_pred_train, y_train, average = 'macro')
f1score = f1_score(y_test, y_pred_test, average= 'macro')
print('f1 train score:', f1score_train)
print('f1 test score:', f1score)

f1 train score: 0.9957318275216104
f1 test score: 0.09019620952825225


In [18]:
#randomforest 

randomforest = RandomForestClassifier(n_estimators = 100, random_state = 42)
randomforest = randomforest.fit(X_train,y_train)
test_pred = randomforest.predict(X_test)
f1score1 = f1_score(test_pred,y_test, average= 'macro')
print('f1 test score:', f1score1)

f1 test score: 0.19021801230544338


In [19]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         afr       1.00      1.00      1.00       583
         eng       1.00      1.00      1.00       615
         nbl       0.99      0.99      0.99       583
         nso       1.00      1.00      1.00       625
         sot       1.00      1.00      1.00       618
         ssw       1.00      1.00      1.00       584
         tsn       1.00      1.00      1.00       598
         tso       1.00      1.00      1.00       561
         ven       1.00      1.00      1.00       634
         xho       1.00      1.00      1.00       609
         zul       0.99      0.99      0.99       590

    accuracy                           1.00      6600
   macro avg       1.00      1.00      1.00      6600
weighted avg       1.00      1.00      1.00      6600



In [20]:
f1score3 = f1_score(y_pred, y_test, average= 'macro')
print('Multinomial f1_score:', f1score3)

Multinomial f1_score: 0.9970937238401433


## Kaggle Submission

In [21]:
X_Kaggle = test_df['text']

In [22]:
kaggle_predict = svc.predict(vect.transform(X_Kaggle))

In [23]:
pred_df = pd.DataFrame(data=test_df['index'], columns=['index']) 
pred_df.insert(1, 'lang_id', kaggle_predict, allow_duplicates=False)

pred_df.head()

,index,lang_id
0,1,tsn
1,2,nbl
2,3,ven
3,4,ssw
4,5,afr


In [24]:
pred_df.to_csv( "TitusO.csv", index=False, quoting=3 )

## CONCLUSION

The best model had an accuracy of 99.7% which is a good score to give conclusive results.